# Line Bot 教學
## 使用 Python LINE Bot SDK 在 Heroku 上架設一個可視覺分析的機器人。

- [官網SDK Python](https://github.com/line/line-bot-sdk-python)

## 在你開始之前，確保您具有以下內容：

- 擁有 Line 帳號
- 擁有 [Heroku](https://www.heroku.com) 帳戶（您可以免費創建一個）

## 建立 Heroku 專案
1. 登入 Heroku 後，
  在 [Heroku](https://dashboard.heroku.com/apps) 頁面中，點選 New -> Create New App
2. 輸入 App name ，然後點擊 Create app
3. 取得網址: (https://<自己的APP_Name>.herokuapp.com)

## 創建 Line Bot 頻道
1. 進入 [Line 控制台](https://developers.line.me/console/)
2. 登入後Create一個新的機器人專案
3. 在'Basic information'填寫下面選項 Channel name, Channel description, Email address
4. 取得Key: Channel secret、Assertion Signing Key， 以及 Massage APP -> Channel access token 備用
5. Message API -> 開啟 use webhook
6. 設置Webhook settings -> 填入Webhook URL: https://\<自己的APP_Name>.herokuapp.com/callback
7. 可關閉預設罐頭回覆訊息在 Message API -> Auto-reply message, Greeting Message 設定成Disable.


## 建立主程式 app.py
- 此教案只做說明，若要執行程式，建議推播到Heroku web app環境後查看Log.
- 程式流程說明:

In [ ]:
#載入套件
from flask import Flask, request, abort, jsonify, make_response
from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import *
import json
import requests
import shutil
import sys
import os

app = Flask(__name__)

- 替換掉下面Token, Webhook key等，由 Line Developers 的專案新增設定的 Line Chot Bot 選單中取得.
- get_web_data : 當我們從Message取得訊息時，只能取到 id 及 type等，例如: type: image, id: 12345678
- 從Line Platform取得圖片、影片、聲音、檔案(https://api-data.line.me/v2/bot/message/{messageId}/content) ，且需要帶入TOKEN才能取得。
- 若要取User上傳內容，例: https://api-data.line.me/v2/bot/message/12345678/content
- 詳細如可以參考:[官網Message API說明](https://developers.line.biz/en/reference/messaging-api/#get-content)


In [ ]:
# Channel Access Token
line_bot_api_Token = '0123456789/NwnXZLq5EzaCZc6IJpbxJxR7chgVpU8LQe6VPau8RGfslcxcWeC4rIGOl606sZsWkkAJmzNn+li/QVHDF9h12zVxeqPbb06Tkapffs4uKgHYepd+TdUQCPnAE0jMVhJqXPbmgdB04t89/1O/w1cDnyilFU='
line_bot_api = LineBotApi(line_bot_api_Token)

# Channel Secret
handler = WebhookHandler('0123456789xx1bd3043f201d3964c26297')
#
#GET web data, 這段程序已經移動到 Azure平台分析，需要圖片資訊時，由後端直接跟 Line Platform，這邊先註記
# getweb = 'https://api.line.me/v2/bot/message/{}/content'

## 監聽所有User傳遞的訊息
- {Body}內可以看到 User 傳遞的結構內容

In [ ]:
# 監聽所有來自 /callback 的 Post Request
@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']
    # get request body as text
    body = request.get_data(as_text=True) #取得所有 user傳遞訊息結構內容，例如 Message: {id, type...}等
    print('body: ',body)
    app.logger.info("Request body: " + body)
    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'



# 取出User TextMessage並進行回應
- 此段為Line Chot Bot python 範例，主要可用於測試是否Chot Bot的架設環境是否已經暢通，如 User <-> Line Platform <-> Haruko Web APP

In [ ]:
# 處理與回應訊息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    print(event.message.text) # User傳遞進來的文字訊息
    message = TextSendMessage(text=event.message.text) # 將 User Text Message封裝
    line_bot_api.reply_message(event.reply_token, message) # 將 TOKEN、封裝好的 Text Message回應給 User

# 在 app.py中只要到此段落，且將下面註解取消掉即可上傳到 Heroku web去啟動 flask.並看Chot bot是否已經可以回應輸入的文字
# 若一切正常則將可以繼續，若不正常就需要檢查設定或是 Heroku Web Log顯示的錯誤內容。
'''
if __name__ == "__main__":
    port = int(os.environ.get('PORT', 5000))
    app.run(host='0.0.0.0', port=port)
'''

# Heroku web上架
1. 下載並安裝 [Heroku CLI](https://devcenter.heroku.com/articles/heroku-cli)、[Git](https://git-scm.com/)
2. 開啟剛剛下載的範例程式碼資料夾，在路徑上輸入 cmd
3. 使用終端或命令行應用程序登錄到 Heroku
    ```shell＝
    heroku login
    ```
4. 初始化 git
    ``` shell=
    $ git config --global user.name <你的名字>
    $ git config --global user.email <你的信箱>
    ```

5. 初始化 git
    ```shell＝
    $ git init
    ```
    注意：僅第一次使用時要輸入

6. 用 git 將資料夾與 heroku 連接
    ```shell
    $ heroku git:remote -a {HEROKU_APP_NAME}
    ```
    注意：{HEROKU_APP_NAME} 是 Heroku 應用的名稱
    
7. 輸入以下指令，將程式碼推上 Heroku，**如果有跳出錯誤請重新輸入**
    ```shell
    git add .
    git commit -m "Add code"
    git push -f heroku master
    ```
    **每當需要更新 Bot 時，請重新輸入上述指令**
    
8. 查看Web Log:
    ```shell
    heroku logs --tail --app {HEROKU_APP_NAME}  #查看Log
    heroku ps:restart web #重啟
    ```
    **程式中的錯誤查看**
    

## 程式檔案解說

> 資料夾裡需含有兩份文件來讓你的程式能在 heroku 上運行

- Procfile：heroku 執行命令，web: {語言} {檔案}，這邊語言為 python，要自動執行的檔案為 app.py，因此我們改成 **web: python app.py**。
- requirements.txt：列出所有用到的套件，heroku 會依據這份文件來安裝需要套件

## Line SDK進階操作
[官方文件](https://github.com/line/line-bot-sdk-python#api)

##### - 平台架構: user -> Line Platform -> Heroku web -> Azure analysis VM --<response>--> Heroku web -> Line Platform -> user

### ImageSendMessage（圖片訊息） < 1MB
```python
message = ImageSendMessage(
    original_content_url='https://example.com/original.jpg',
    preview_image_url='https://example.com/preview.jpg'
)
line_bot_api.reply_message(event.reply_token, message)
```
如下面程式碼:

In [ ]:
# 影像訊息的處理
@handler.add(MessageEvent, message=ImageMessage) #這段只處理 ImageMessage
def get_message_content(event):
    line_key = {
        "id": "",
        "bearer": ""
    }
    headers = {
        "Content-Type": "application/json"
    }
    print('EVENT MSG: ', event.message)
    print('message type: ', event.message.type)
    print('Content_provider', event.message.content_provider)
    print("id: ", event.message.id)
    line_key['id'] = event.message.id
    line_key['bearer'] = line_bot_api_Token
    try:
        #通知 Azure CPS server，且帶入 id, bearer key等，並開始進行分析
        res = requests.post('http://countpersonvm.australiacentral.cloudapp.azure.com/notification',
                          data = json.dumps(line_key), headers=headers, verify=False, timeout = 30)
        response = json.loads(res.text)
        filename = str(response["filename"])
        #print(response)
        if str(response["status"]) == "200": #圖片分析成功
            #print('200')
            #建立 urllink，可從 Azure的網址可以取得圖片，是已分析結果的圖片網址，並將 filename串接在後方
            urllink = 'http://countpersonvm.australiacentral.cloudapp.azure.com/show/' + filename
            headers = {"Content-Type":"image/png"}
            r = requests.get(urllink, headers=headers, stream=True) #去Azure取分析圖片

            if os.path.isdir(os.path.join(os.getcwd(),'static', 'model')):
                shutil.rmtree(os.path.join(os.getcwd(),'static','model'))
                os.mkdir(os.path.join(os.getcwd(), 'static', 'model'))
            else:
                os.mkdir(os.path.join(os.getcwd(),'static', 'model'))

            if r.status_code == 200: #若取得分析圖片，並存檔
                print('[Save Pass]: ', str(response["filename"]))
                with open(os.path.join(os.getcwd(),'static', 'model', filename), 'wb') as f:
                    r.raw.decode_content = True
                    shutil.copyfileobj(r.raw, f)
                    
            # 建立 urllink連結，這是提供給 Line Platform 取用來且回應給 user，
            # 由於 Azure的 https憑證需要購買，因此使用 heroku的 https網址進行取圖片
            urllink = 'https://linechtbot3probe.herokuapp.com/show/' + filename
            result = "\n".join(str(response["all_text"]).split(','))
            
        else: #圖片分析失敗就回應錯誤
            filename = 'server_error.JPG'
            urllink = 'https://linechtbot3probe.herokuapp.com/show_error/' + filename
            #print('伺服器異常或稍後重新上傳')
            result = "伺服器異常或稍後重新上傳"
        return result, urllink, filename

    except Exception as e: #圖片分析失敗或伺服器異常就回應錯誤
        print("[Exception] {}: {}".format(type(e).__name__, e))
        print("[Exception] Error on line {}.".format(sys.exc_info()[-1].tb_lineno))
        filename = 'server_error.JPG'
        urllink = 'https://linechtbot3probe.herokuapp.com/show_error/' + filename
        #print('伺服器異常或稍後重新上傳')
        result = "伺服器異常或稍後重新上傳"
        return result, urllink, filename

    finally: #封裝所有結果及訊息回應給 Line Platform
        message = TextSendMessage(text=str(result))
        image = ImageSendMessage(
            original_content_url=urllink,
            preview_image_url=urllink)
        line_bot_api.reply_message(event.reply_token, [image, message])

### 取圖\<heroku>/show>及發生錯誤\<heroku>/show_photo_error
- 在上面有呼叫兩個網址，分別是 \<heroku>/show 以及 \<heroku>/show_error，因此需要設定網址回應呼叫的程序及內容。

In [ ]:
# 取圖網址，若有前端來GET
@app.route('/show/<string:filename>', methods=['GET'])
def show_photo(filename):
    try:
        file_dir = os.path.join(os.getcwd(), 'static', 'model') #圖檔目錄
        if request.method == 'GET':
            if filename is None: #檔案不存在就顯示'無此檔案'
                errorinfo = {"error": 1001, "msg": "無此檔案"}
                return jsonify(errorinfo)
            else: # 若檔案存在於目錄，則讀取檔案後並傳送回去
                #print('Show: ',os.path.join(file_dir, str(filename)))
                image_data = open(os.path.join(file_dir, str(filename)), "rb").read()
                response = make_response(image_data)
                response.headers['Content-Type'] = 'image/png'
                return response
        else:
            pass
    except FileNotFoundError:
        errorinfo = {"error": 1001, "msg": "無此檔案"}
        return jsonify(errorinfo)

# 錯誤回應網址，若有前端來GET
@app.route('/show_error/<string:filename>', methods=['GET'])
def show_photo_error(filename):
    try:
        file_dir = os.path.join(os.getcwd(), 'static','error') #圖檔目錄
        if request.method == 'GET':
            if filename is None:
                errorinfo = {"error": 1001, "msg": "無此檔案"}
                return jsonify(errorinfo)
            else: # 若檔案存在於目錄，則讀取檔案後並傳送回去
                image_data = open(os.path.join(file_dir, str(filename)), "rb").read()
                response = make_response(image_data)
                response.headers['Content-Type'] = 'image/png'
                return response
        else:
            pass
    except FileNotFoundError:
        errorinfo = {"error": 1001, "msg": "無此檔案"}
        return jsonify(errorinfo)

In [ ]:
# flask 啟動
if __name__ == "__main__":
    port = int(os.environ.get('PORT', 5000))
    app.run(host='0.0.0.0', port=port)

### Rich Menu 設定、刪除、查詢、更新請參考教案檔案: Line_Rich_Menu.ipynb
### Azure 分析平台範例說明請參考 Line_image_upload.ipynb (完成方案請參考Git )